## Exploratory Analysis

That's it for some preliminary cleaning. Don't worry, there will be more. Let's start to look in a bit more detail at the data, though. In this section, we're going to start to write some code that's typical for day-to-day data cleaning tasks.

In [65]:
from load_data import dta
import pandas as pd
import numpy as np

We can use `info` to get some high-level information about the data.

In [2]:
dta.head()

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,violations,zip
inspection_id,,,,,,,,,,,,,,,
1965287,5255 W MADISON ST,RED SNAPPER FISH CHICKEN & PIZZA,CHICAGO,RED SNAPPER FISH CHICKEN & PIZZA,Restaurant,2016-09-26,Canvass,41.880237,1991820.0,-87.757220,Pass w/ Conditions,Risk 1 (High),IL,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",60644
1329698,5958 W DIVERSEY AVE,TAQUERIA MORELOS,CHICAGO,TAQUERIA MORELOS,Restaurant,2014-02-06,Canvass,41.931250,2099479.0,-87.775907,Pass,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60639
470787,5400-5402 N CLARK ST,HAMBURGER MARY'S/MARY'S REC ROOM,CHICAGO,HAMBURGER MARY'S CHICAGO/MARY'S REC ROOM,Restaurant,2010-12-03,SFP,41.979884,1933748.0,-87.668429,Fail,Risk 1 (High),IL,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA...",60640
68091,2804 N CLARK ST,Wells Street Popcorn,CHICAGO,Wells Street Popcorn,Restaurant,2010-02-01,Canvass,41.932921,1954774.0,-87.645155,Pass,Risk 2 (Medium),IL,NaN,60657
1335320,2409 N WESTERN AVE,CHICAGO CUPCAKE,CHICAGO,CHICAGO CUPCAKE LLC.,Mobile Food Dispenser,2013-05-03,License Re-Inspection,41.925218,2232391.0,-87.687507,Fail,Risk 3 (Low),IL,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,60647


In [3]:
dta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 1965287 to 413268
Data columns (total 15 columns):
address            25000 non-null object
aka_name           24709 non-null object
city               24981 non-null object
dba_name           25000 non-null object
facility_type      24787 non-null category
inspection_date    25000 non-null datetime64[ns]
inspection_type    25000 non-null category
latitude           24865 non-null float64
license_           24996 non-null float64
longitude          24865 non-null float64
results            25000 non-null category
risk               24995 non-null category
state              24994 non-null object
violations         23908 non-null object
zip                24990 non-null object
dtypes: category(4), datetime64[ns](1), float64(3), object(7)
memory usage: 2.4+ MB


And `describe` goes into a bit more detail for the *numeric* types, of which we don't have many here.

In [4]:
dta.describe()

,latitude,license_,longitude
count,24865.000000,2.499600e+04,24865.000000
mean,41.879734,1.475028e+06,-87.675890
std,0.081968,8.912679e+05,0.058372
min,41.644670,0.000000e+00,-87.914428
25%,41.828059,1.000632e+06,-87.707209
50%,41.891528,1.884173e+06,-87.666377
75%,41.939792,2.134724e+06,-87.634533
max,42.021064,8.700606e+06,-87.525125


We could do the same for the categorical types.

In [5]:
dta.select_dtypes(['category']).describe()

,facility_type,inspection_type,results,risk
count,24787,25000,25000,24995
unique,226,40,7,4
top,Restaurant,Canvass,Pass,Risk 1 (High)
freq,16954,12487,15915,18249


### GroupBy

Now, let's ask the most obvious question. Which are the best and the worst restaurants? We'll want to use pandas `GroupBy` functionality to implement the `split-apply-combine` pattern.

The idea here is that we **split** the data by some key or set of keys then **apply** a function to each group and then **combine** the outputs back into a single DataFrame.

First, let's see how many result categories there are. We can use `value_counts` to answer this question. 

In [6]:
with pd.option_context("max.rows", 10):
    print(dta.results.value_counts())

Pass                    15915
Fail                     5760
Pass w/ Conditions       2931
Out of Business           246
No Entry                  107
Not Ready                  40
Business Not Located        1
Name: results, dtype: int64


Ok, let's group on the inspection `results` column and see who the best and worst are.

When we call the `groupby` method we get back a `DataFrameGroupBy` object.

In [7]:
grouper = dta.groupby(dta.results)

You can access the variables on this object, the same as a DataFrame, and any code called will execute within the groups.

In [8]:
result = grouper.dba_name.value_counts()

The result is a Series with a `MultiIndex`.

In [9]:
result

results               dba_name                   
Business Not Located  CITGO SUPER WASH & GAS          1
Fail                  SUBWAY                         42
                      DUNKIN DONUTS                  33
                      MCDONALD'S                     20
                      HAROLD'S CHICKEN SHACK         14
                                                     ..
Pass w/ Conditions    ZELLA                           1
                      ZOOM PIZZA FACTORY, INC.        1
                      ZUZU WRAPS INC                  1
                      Zaytune Mediterranean Grill     1
                      subway restaurant 1             1
Name: dba_name, Length: 16459, dtype: int64

In [10]:
result.index.names

FrozenList(['results', 'dba_name'])

In [11]:
result.to_frame().head(n = 30)

dba_name
results              dba_name                                     
Business Not Located CITGO SUPER WASH & GAS                      1
Fail                 SUBWAY                                     42
                     DUNKIN DONUTS                              33
                     MCDONALD'S                                 20
                     HAROLD'S CHICKEN SHACK                     14
                     CHIPOTLE MEXICAN GRILL                     11
                     DUNKIN DONUTS / BASKIN ROBBINS             10
                     7-ELEVEN                                    9
                     MCDONALDS                                   9
                     POTBELLY SANDWICH WORKS                     9
                     POTBELLY SANDWICH WORKS LLC                 9
                     JIMMY JOHNS                                 8
                     LAS ISLAS MARIAS                            8
                     KFC                                         7
                     PIZZA HUT                                   7
                     BAR LOUIE                                   6
                     CORNER BAKERY CAFE                          6
                     DUNKIN DONUTS/BASKIN ROBBINS                6
                     EL TACONAZO RESTAURANT                      6
                     FRESHII                                     6
                     HALSTED STREET DELI                         6
                     OLD COUNTRY BUFFET                          6
                     SHARKS FISH & CHICKEN                       6
                     AMAZING GRACE DAYCARE CENTER                5
                     ANTON DVORAK ELEMENTARY                     5
                     CHURCH'S CHICKEN # 992                      5
                     CITGO                                       5
                     Congress Pizza                              5
                     DOMINO'S PIZZA                              5
                     EL CHILE RESTAURANT & PANCAKE HOUSE         5

An alternative way: use the `size()` method.

In [12]:
grouper1 = dta.groupby(['results', 'dba_name'])
grouper1.size().sort_values(ascending=False)

results               dba_name               
Pass                  SUBWAY                     219
                      DUNKIN DONUTS              118
Pass w/ Conditions    SUBWAY                      59
Pass                  MCDONALD'S                  42
Fail                  SUBWAY                      42
                                                ... 
Pass                  MC CANN DAY CARE CENTER      1
                      MAZE FOOD & LIQUOR           1
                      MAYS ELEMENTARY              1
                      MAYRA'S FAMILY CATERING      1
Business Not Located  CITGO SUPER WASH & GAS       1
Length: 16459, dtype: int64

We can index on the first element in a `MultiIndex` using square brackets and then use `sort_values` to find those restaurants that had a result of Fail the most.

In [13]:
with pd.option_context('max.rows', 15):
    print(result["Fail"].sort_values(ascending=False))

dba_name
SUBWAY                    42
DUNKIN DONUTS             33
MCDONALD'S                20
HAROLD'S CHICKEN SHACK    14
CHIPOTLE MEXICAN GRILL    11
                          ..
MERCER 113                 1
MELLOS PEANUT COMPANY      1
MEHRAB SUPER MARKET        1
MEERATH KABAB              1
HALSTED FOOD               1
Name: dba_name, Length: 4424, dtype: int64


Take a closer look above. Looks like we have some more data cleaning to do.

In [14]:
with pd.option_context('max.rows', 15):
    print(result["Pass"].sort_values(ascending=False))

dba_name
SUBWAY                  219
DUNKIN DONUTS           118
MCDONALD'S               42
7-ELEVEN                 38
AU BON PAIN              29
                       ... 
KASEY'S TAVERN            1
KARYN'S ON GREEN          1
KARLA'S KITCHEN INC.      1
KARLA'S KITCHEN           1
CITY DOUGH                1
Name: dba_name, Length: 9328, dtype: int64


This is probably not the right way to think about this since there are many more Subways than local establishments.

We could instead look at the ratio of Fail to Pass, though, of course, this isn't perfect either. 

Sometimes, it's not *always* obvious how to go about computing things that you want to compute. The `get_group` method allows you to pull out one of the split DataFrames and try your apply function on it.

## Exercise

GroupBy the `dba_name`. Use `get_group` to pull out the "MCDONALD'S" group. Write a function that calculates the relative number of Fail to Pass for this group. Run this function on the McDonald's group.

In [16]:
# Type your solution here
def calculate_fail_to_pass(dat):
    fail_to_pass_ratio = dat['Fail']/dat['Pass']
    return fail_to_pass_ratio

my_data = dta.groupby(['dba_name']).get_group("MCDONALD'S").results.value_counts()
calculate_fail_to_pass(my_data)

0.47619047619047616

In [ ]:
%load solutions/get_group.py

# And we see McDonald's failed 50% as many inspections as it Passed.

We can run this on everything, but it's going to be a little slow. Let's look at another way to approach this problem.

Here group by *both* the inspection results and the DBA name. Then we ask for the `size` of each one of these groups.

In [17]:
result = dta.groupby((dta.results, dta.dba_name)).size()

/Users/firefreezing/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


In [18]:
result

results               dba_name                        
Business Not Located  CITGO SUPER WASH & GAS              1
Fail                  #1 WOK N ROLL                       2
                      'C' A CATERING AND EVENT COMPANY    1
                      10 PIN  BOWLING LOUNGE              1
                      1104 WABASH                         1
                                                         ..
Pass w/ Conditions    ZOCALO                              2
                      ZOOM PIZZA FACTORY, INC.            1
                      ZUZU WRAPS INC                      1
                      Zaytune Mediterranean Grill         1
                      subway restaurant 1                 1
Length: 16459, dtype: int64

We can use the `div` method to divide these for us. As you can see the indices don't line up, but we don't have to worry about it. Pandas take care of index alignment for us.

In [19]:
result["Fail"]

dba_name
#1 WOK N ROLL                       2
'C' A CATERING AND EVENT COMPANY    1
10 PIN  BOWLING LOUNGE              1
1104 WABASH                         1
111 TH FOOD & CELLULAR, INC         1
                                   ..
ZOZO                                1
ZULLO'S INC.                        1
iCREAM                              1
mcdonalds                           1
stockton                            1
Length: 4424, dtype: int64

In [20]:
result["Pass"]

dba_name
#1 CHOP SUEY                        3
#1 WOK N ROLL                       2
'C' A CATERING AND EVENT COMPANY    2
1 JACKPOT MINI MART, INC.           2
1 STOP & SHOP INC.                  1
                                   ..
elinas cafe                         1
mEAT ON CHESTNUT                    1
markos tacos&cafe                   1
mr.daniel's                         1
stockton                            1
Length: 9328, dtype: int64

In [21]:
ratio = result["Fail"].div(result["Pass"])
ratio.sort_values(ascending=False, inplace=True)
ratio

dba_name
EL CHILE RESTAURANT & PANCAKE HOUSE    5.0
AMAZING GRACE DAYCARE CENTER           5.0
THE NEW THREE HAPPINESS, INC           5.0
CHURCH'S CHICKEN # 992                 5.0
HEMA'S KITCHEN                         5.0
                                      ... 
iCREAM                                 NaN
mEAT ON CHESTNUT                       NaN
markos tacos&cafe                      NaN
mcdonalds                              NaN
mr.daniel's                            NaN
Length: 11117, dtype: float64

We have a lot of `NaN`s in the results from division-by-zero. We can drop those with a call to `dropna`. Also note that pandas lets you decide whether to treat `inf` as an NA.

In [22]:
with pd.option_context("use_inf_as_null", True,
                       "max.rows", 15):
    print(ratio.dropna())

dba_name
EL CHILE RESTAURANT & PANCAKE HOUSE    5.000000
AMAZING GRACE DAYCARE CENTER           5.000000
THE NEW THREE HAPPINESS, INC           5.000000
CHURCH'S CHICKEN # 992                 5.000000
HEMA'S KITCHEN                         5.000000
                                         ...   
MACY'S                                 0.090909
COCULA RESTAURANT                      0.090909
HOST INTERNATIONAL INC                 0.083333
TACO BURRITO KING                      0.076923
TRIPLE A SERVICES, INC.                0.071429
Length: 2635, dtype: float64


We might still not be wholly satisfied with our rules around comparisons here. First, we're looking at restaurant names not particular establishments. What does the distribution of inspection visits for establishments look like.

In [23]:
with pd.option_context("max.rows", 10):
    grouper = dta.groupby((dta.address, dta.dba_name))
    print(grouper.size().describe())

count    13444.000000
mean         1.859566
std          1.215183
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         15.000000
dtype: float64


/Users/firefreezing/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  


Ok, let's make things a little more challenging. Let's see what the Fail:Pass ratio is for restaurants with at least 3 visits that involved a high risk level. 


Now we're starting to get into some much more powerful pandas constructs.

In [24]:
visited = (dta.query("risk == 'Risk 1 (High)'")
           .groupby(('address', 'dba_name'))
           .size()
           .rename('n_visits')
           .reset_index()  # make into a DataFrame
           .query("n_visits >= 4"))

visited

/Users/firefreezing/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  


,address,dba_name,n_visits
0,1 E 113TH ST,"V & J DAY CARE CENTER, INC",6
2,1 E JACKSON BLVD,Chartwells @ DePaul University,5
4,1 E WACKER DR,MCCORMICK & SCHMICK'S SEAFOOD RESTAURANT,4
18,1 W WACKER DR,RENAISSANCE CHICAGO HOTEL,4
26,10 N DEARBORN ST,SOPRAFFINA,4
37,10 W HUBBARD ST,VERMILION,5
40,100 E CHESTNUT ST,MIKE DITKA'S RESTAURANT,5
47,100 N LA SALLE ST,Qdoba Mexican Eats #2621,4
52,100 W 111TH ST,JACK' S FAMOUS WINGS,4
53,100 W 87TH ST,GRAND CHINESE KITCHEN,4


Let's unpack this. The first thing to note is how this code is organized. Each one of these methods return a pandas data structure on which we call the next method. This is called **method chaining**. We use the same trick seen above to split strings across lines to split several method calls by including the code between `()`.

Next, we see several new methods. The first is **query**. When subsetting a DataFrame we have a few options. As we save above, we can index a DataFrame using integers. Likewise, we could pass an object of booleans as well.

In [25]:
dta.risk == "Risk 1 (High)"

inspection_id
1965287     True
1329698     True
470787      True
68091      False
1335320    False
           ...  
2059771     True
1965378     True
1490395    False
1326565    False
413268      True
Name: risk, Length: 25000, dtype: bool

In [26]:
dta.loc[dta.risk == "Risk 1 (High)"]

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,violations,zip
inspection_id,,,,,,,,,,,,,,,
1965287,5255 W MADISON ST,RED SNAPPER FISH CHICKEN & PIZZA,CHICAGO,RED SNAPPER FISH CHICKEN & PIZZA,Restaurant,2016-09-26,Canvass,41.880237,1991820.0,-87.757220,Pass w/ Conditions,Risk 1 (High),IL,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",60644
1329698,5958 W DIVERSEY AVE,TAQUERIA MORELOS,CHICAGO,TAQUERIA MORELOS,Restaurant,2014-02-06,Canvass,41.931250,2099479.0,-87.775907,Pass,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60639
470787,5400-5402 N CLARK ST,HAMBURGER MARY'S/MARY'S REC ROOM,CHICAGO,HAMBURGER MARY'S CHICAGO/MARY'S REC ROOM,Restaurant,2010-12-03,SFP,41.979884,1933748.0,-87.668429,Fail,Risk 1 (High),IL,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA...",60640
1285582,3201-3203 W ARMITAGE AVE,STREET SIDE CAFE,CHICAGO,STREET SIDE CAFE,Restaurant,2012-09-27,Canvass,41.917291,84648.0,-87.707052,Pass w/ Conditions,Risk 1 (High),IL,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",60647
74468,4445 S Drexel (900E),KING COLLEGE PREP HIGH SCHOOL,CHICAGO,KING SELECTIVE ENROL.H/S,School,2010-05-26,Canvass,41.813695,46371.0,-87.603838,Pass,Risk 1 (High),IL,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,60653
1522863,3958 N NARRAGANSETT AVE,MR ZEE'S,CHICAGO,"MR ZEES, INC",Restaurant,2015-02-10,Canvass,41.952792,49683.0,-87.786701,Pass,Risk 1 (High),IL,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,60634
1365288,10-20 E DELAWARE ST,LITTLE MARKET BRASSIERE,CHICAGO,LITTLE MARKET BRASSIERE,Restaurant,2013-09-26,Canvass Re-Inspection,41.899255,2157256.0,-87.627835,Pass,Risk 1 (High),IL,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,60611
1453434,81 E VAN BUREN ST,THE CHICAGO CLUB,CHICAGO,THE CHICAGO CLUB,Restaurant,2014-07-21,Canvass,41.876897,36751.0,-87.624651,Pass,Risk 1 (High),IL,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",60605
1978957,3042 N BROADWAY,STELLA'S DINER,CHICAGO,STELLA'S DINER,Restaurant,2017-01-05,Complaint,41.937665,3769.0,-87.644468,Pass,Risk 1 (High),IL,45. FOOD HANDLER REQUIREMENTS MET - Comments: ...,60657


Always using indexing can be verbose, however. You may need compound statements, for example.

In [27]:
dta.loc[(dta.risk == "Risk 1 (High)") | (dta.risk == "Risk 1 (Medium)")].head()

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,violations,zip
inspection_id,,,,,,,,,,,,,,,
1965287,5255 W MADISON ST,RED SNAPPER FISH CHICKEN & PIZZA,CHICAGO,RED SNAPPER FISH CHICKEN & PIZZA,Restaurant,2016-09-26,Canvass,41.880237,1991820.0,-87.757220,Pass w/ Conditions,Risk 1 (High),IL,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",60644
1329698,5958 W DIVERSEY AVE,TAQUERIA MORELOS,CHICAGO,TAQUERIA MORELOS,Restaurant,2014-02-06,Canvass,41.931250,2099479.0,-87.775907,Pass,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60639
470787,5400-5402 N CLARK ST,HAMBURGER MARY'S/MARY'S REC ROOM,CHICAGO,HAMBURGER MARY'S CHICAGO/MARY'S REC ROOM,Restaurant,2010-12-03,SFP,41.979884,1933748.0,-87.668429,Fail,Risk 1 (High),IL,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA...",60640
1285582,3201-3203 W ARMITAGE AVE,STREET SIDE CAFE,CHICAGO,STREET SIDE CAFE,Restaurant,2012-09-27,Canvass,41.917291,84648.0,-87.707052,Pass w/ Conditions,Risk 1 (High),IL,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",60647
74468,4445 S Drexel (900E),KING COLLEGE PREP HIGH SCHOOL,CHICAGO,KING SELECTIVE ENROL.H/S,School,2010-05-26,Canvass,41.813695,46371.0,-87.603838,Pass,Risk 1 (High),IL,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,60653


Instead, by using query we could write the following, which is slightly easier.

In [28]:
dta.query("(risk == 'Risk 1 (High)') | (risk == 'Risk 1 (Medium)')").head()

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,violations,zip
inspection_id,,,,,,,,,,,,,,,
1965287,5255 W MADISON ST,RED SNAPPER FISH CHICKEN & PIZZA,CHICAGO,RED SNAPPER FISH CHICKEN & PIZZA,Restaurant,2016-09-26,Canvass,41.880237,1991820.0,-87.757220,Pass w/ Conditions,Risk 1 (High),IL,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",60644
1329698,5958 W DIVERSEY AVE,TAQUERIA MORELOS,CHICAGO,TAQUERIA MORELOS,Restaurant,2014-02-06,Canvass,41.931250,2099479.0,-87.775907,Pass,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60639
470787,5400-5402 N CLARK ST,HAMBURGER MARY'S/MARY'S REC ROOM,CHICAGO,HAMBURGER MARY'S CHICAGO/MARY'S REC ROOM,Restaurant,2010-12-03,SFP,41.979884,1933748.0,-87.668429,Fail,Risk 1 (High),IL,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA...",60640
1285582,3201-3203 W ARMITAGE AVE,STREET SIDE CAFE,CHICAGO,STREET SIDE CAFE,Restaurant,2012-09-27,Canvass,41.917291,84648.0,-87.707052,Pass w/ Conditions,Risk 1 (High),IL,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",60647
74468,4445 S Drexel (900E),KING COLLEGE PREP HIGH SCHOOL,CHICAGO,KING SELECTIVE ENROL.H/S,School,2010-05-26,Canvass,41.813695,46371.0,-87.603838,Pass,Risk 1 (High),IL,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,60653


## Exercise

Use `query` to find the visits that are to restaurants and that are complaint re-inspections.

In [29]:
dta.groupby(dta.inspection_type).size().head()

inspection_type
Business Not Located         1
CANVASS/SPECIAL EVENT        1
CHANGED COURT DATE           1
Canvass                  12487
Canvass Re-Inspection     3181
dtype: int64

In [30]:
# Type your solution here
dta.query("facility_type == 'Restaurant' & inspection_type == 'Complaint Re-Inspection'")

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,violations,zip
inspection_id,,,,,,,,,,,,,,,
614430,201 W MADISON ST,DUNKIN DONUTS,CHICAGO,DUNKIN DONUTS,Restaurant,2011-08-22,Complaint Re-Inspection,41.881835,1742236.0,-87.633897,Pass,Risk 2 (Medium),IL,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,60606
2081698,1524 W 115TH ST,I-57 RIB HOUSE,CHICAGO,I-57 RIB HOUSE,Restaurant,2017-09-07,Complaint Re-Inspection,41.684938,58149.0,-87.661123,Pass,Risk 1 (High),IL,19. OUTSIDE GARBAGE WASTE GREASE AND STORAGE A...,60643
1385475,5542 W NORTH AVE,SUNSUB & PHILLY'S,CHICAGO,SUNSUB & PHILLY'S,Restaurant,2014-07-18,Complaint Re-Inspection,41.909498,2129537.0,-87.764807,Pass w/ Conditions,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60639
1946225,4623-4627 N BROADWAY,IYANZE,CHICAGO,IYANZE,Restaurant,2016-07-07,Complaint Re-Inspection,41.966063,1909713.0,-87.657734,Pass,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60640
666300,225 S CANAL ST,KELLY'S CAJUN GRILL,CHICAGO,KCG OF CHICAGO UNION STATION,Restaurant,2012-01-03,Complaint Re-Inspection,41.878531,69186.0,-87.639427,Fail,Risk 1 (High),IL,"9. WATER SOURCE: SAFE, HOT & COLD UNDER CITY P...",60606
680079,1168-1172 E 55TH ST,THE WOODLAWN TAP,CHICAGO,THE WOODLAWN TAP,Restaurant,2012-03-09,Complaint Re-Inspection,41.795173,79075.0,-87.596851,Pass,Risk 2 (Medium),IL,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",60615
233395,1919 N CICERO AVE,QUITE FRANKLY,CHICAGO,QUITE FRANKLY,Restaurant,2010-04-29,Complaint Re-Inspection,41.915710,1355963.0,-87.745908,Pass,Risk 2 (Medium),IL,"26. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, P...",60639
1096558,520 S MICHIGAN AVE,THE CONGRESS HOTEL(TAVERN HALL-RAFAEL),CHICAGO,THE CONGRESS HOTEL,Restaurant,2012-10-02,Complaint Re-Inspection,41.875260,49248.0,-87.624397,Pass,Risk 1 (High),IL,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,60605
1386529,1359 W 79TH ST,NaN,CHICAGO,I & O FOOD MART INC,Restaurant,2015-10-29,Complaint Re-Inspection,41.750317,2363441.0,-87.658527,Pass w/ Conditions,Risk 1 (High),IL,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",60620


In [ ]:
%load solutions/query.py

```python
visited = (dta.query("risk == 'Risk 1 (High)'")
           .groupby(('address', 'dba_name'))
           .size()
           .rename('n_visits')  # size returns a nameless series
           .reset_index()  # make into a DataFrame
           .query("n_visits >= 4"))
```


The next new method is the **rename** method. We use this to rename the unnamed Series returned by `size`.

Finally, we filter on restaurants with 4 or more total visits.

The final piece is computing the Fail:Pass ratio of these restaurants. To do this, we need to take the output we've created `visited` and line our original data up with these addresses and DBA names.

In [31]:
visited.head()

,address,dba_name,n_visits
0,1 E 113TH ST,"V & J DAY CARE CENTER, INC",6
2,1 E JACKSON BLVD,Chartwells @ DePaul University,5
4,1 E WACKER DR,MCCORMICK & SCHMICK'S SEAFOOD RESTAURANT,4
18,1 W WACKER DR,RENAISSANCE CHICAGO HOTEL,4
26,10 N DEARBORN ST,SOPRAFFINA,4


We can do this by using the **merge** method. Merge allows us to make two pandas DataFrames into a single DataFrame. By default, the `merge` method will join together two DataFrames on common columns, using an inner join method (a set intersection).

In [32]:
merged_visits = visited.merge(dta)
merged_visits.head()

,address,dba_name,n_visits,aka_name,city,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,violations,zip
0,1 E 113TH ST,"V & J DAY CARE CENTER, INC",6,"V & J DAY CARE CENTER, INC",CHICAGO,Daycare Above and Under 2 Years,2010-10-08,Canvass,41.688848,1800876.0,-87.622894,Pass,Risk 1 (High),IL,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,60628
1,1 E 113TH ST,"V & J DAY CARE CENTER, INC",6,"V & J DAY CARE CENTER, INC",CHICAGO,Daycare Above and Under 2 Years,2015-09-22,License,41.688848,2215538.0,-87.622894,Fail,Risk 1 (High),IL,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,60628
2,1 E 113TH ST,"V & J DAY CARE CENTER, INC",6,"V & J DAY CARE CENTER, INC",CHICAGO,Daycare Above and Under 2 Years,2010-10-08,Canvass,41.688848,21702.0,-87.622894,Pass,Risk 1 (High),IL,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",60628
3,1 E 113TH ST,"V & J DAY CARE CENTER, INC",6,"V & J DAY CARE CENTER, INC",CHICAGO,Daycare Above and Under 2 Years,2012-01-20,Canvass Re-Inspection,41.688848,21702.0,-87.622894,Pass,Risk 1 (High),IL,19. OUTSIDE GARBAGE WASTE GREASE AND STORAGE A...,60628
4,1 E 113TH ST,"V & J DAY CARE CENTER, INC",6,"V & J DAY CARE CENTER, INC",CHICAGO,Daycare Above and Under 2 Years,2012-01-13,Canvass,41.688848,1800876.0,-87.622894,Fail,Risk 1 (High),IL,19. OUTSIDE GARBAGE WASTE GREASE AND STORAGE A...,60628


In [33]:
merged_visits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5110 entries, 0 to 5109
Data columns (total 16 columns):
address            5110 non-null object
dba_name           5110 non-null object
n_visits           5110 non-null int64
aka_name           5106 non-null object
city               5110 non-null object
facility_type      5107 non-null category
inspection_date    5110 non-null datetime64[ns]
inspection_type    5110 non-null category
latitude           5090 non-null float64
license_           5110 non-null float64
longitude          5090 non-null float64
results            5110 non-null category
risk               5110 non-null category
state              5110 non-null object
violations         5008 non-null object
zip                5110 non-null object
dtypes: category(4), datetime64[ns](1), float64(3), int64(1), object(7)
memory usage: 557.8+ KB


Now, we need to redo the analysis we started above for McDonald's. We take these merged DataFrames, group them by the inspection results, the address, and the DBA name and ask for the size of each group.

In [34]:
(merged_visits
 .groupby(('results', 'address', 'dba_name'))
 .size()).head()

/Users/firefreezing/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  


results  address            dba_name                                
Fail     1 E 113TH ST       V & J DAY CARE CENTER, INC                  2
         1 E JACKSON BLVD   Chartwells @ DePaul University              1
         1 E WACKER DR      MCCORMICK & SCHMICK'S SEAFOOD RESTAURANT    2
         10 N DEARBORN ST   SOPRAFFINA                                  1
         10 W HUBBARD ST    VERMILION                                   1
dtype: int64

## Exercise

Take this result and **pipe** it (using the `pipe` method) to a function that computes Fail/Pass. Make sure your result does not have any missing values, and sort it such that those with the highest Fail/Pass ratios are highest.

The **pipe** method allows for including user-defined functions in method chains. It takes the output of the thing on the left, and passes it to the thing on the right.

In [ ]:
# Type your solution here
(merged_visits
 .groupby(('results', 'address', 'dba_name'))
 .size()
 .rename('n_fails')
 .reset_index()
 .query("results == 'Fail'")
 .merge(visited)
 .pipe(lambda dat: dat.n_fails/dat.n_visits)
 .sort_values(ascending=False)
 .dropna())

In [ ]:
# %load solutions/pipe_results.py
 (merged_visits.pipe(lambda df: df['Fail'].div(df['Pass']))
  .dropna()
  .sort_values(ascending=False))


## Exercise

Now, take everything that we've done above, from `dta` to this final result, and put it together into a single method chain.

In [ ]:
# Type your solution here

In [ ]:
%load solutions/complete_chain.py

Often, I'll try to place a method chain to get the data ready for more exploratory work at the top of a notebook, so I can proceed with any analyses.

Now, let's go back to our original data and add in the unstacked violations to the information that's unchanging. Recall from the previous notebook that we unstack the violations as follows. There are two new things to note here though. We add in a `to_frame` method to turn the unstacked Series into a DataFrame, and we `rename` the unnamed column in the resulting DataFrame back to `violations`.

In [49]:
(dta.violations
 .str.split("|", expand=True)
 .unstack()
 .dropna()
 .reset_index(level=0, drop=True)
 .str.strip()
 .rename('violations')
 .to_frame()
 .sort_values('inspection_id'))

,violations
inspection_id,
44249,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
44249,"41. PREMISES MAINTAINED FREE OF LITTER, UNNECE..."
44249,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR..."
44249,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
44249,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO..."
44259,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
44259,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...
44262,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR..."
44262,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...


First, we need to drop the violations from the original DataFrame, then we need to merge it with the unstacked violations Series that we created before. You can use `drop` to do this.

In [48]:
dta.drop(["violations"], axis='columns').head()

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,zip
inspection_id,,,,,,,,,,,,,,
1965287,5255 W MADISON ST,RED SNAPPER FISH CHICKEN & PIZZA,CHICAGO,RED SNAPPER FISH CHICKEN & PIZZA,Restaurant,2016-09-26,Canvass,41.880237,1991820.0,-87.757220,Pass w/ Conditions,Risk 1 (High),IL,60644
1329698,5958 W DIVERSEY AVE,TAQUERIA MORELOS,CHICAGO,TAQUERIA MORELOS,Restaurant,2014-02-06,Canvass,41.931250,2099479.0,-87.775907,Pass,Risk 1 (High),IL,60639
470787,5400-5402 N CLARK ST,HAMBURGER MARY'S/MARY'S REC ROOM,CHICAGO,HAMBURGER MARY'S CHICAGO/MARY'S REC ROOM,Restaurant,2010-12-03,SFP,41.979884,1933748.0,-87.668429,Fail,Risk 1 (High),IL,60640
68091,2804 N CLARK ST,Wells Street Popcorn,CHICAGO,Wells Street Popcorn,Restaurant,2010-02-01,Canvass,41.932921,1954774.0,-87.645155,Pass,Risk 2 (Medium),IL,60657
1335320,2409 N WESTERN AVE,CHICAGO CUPCAKE,CHICAGO,CHICAGO CUPCAKE LLC.,Mobile Food Dispenser,2013-05-03,License Re-Inspection,41.925218,2232391.0,-87.687507,Fail,Risk 3 (Low),IL,60647


## Exercise

Drop the original violations from `dta`, and **join** this to the unstacked violations as computed above. You'll probably want to use a **right join**.

We use **join** here rather than **merge**. Join uses merge under the hood but conveniently allows us to join on the indices of the two DataFrames by default. One other difference is that join uses an inner merge by default, but that's not what we want here. Since we drop the null violations on the right-hand side DataFrame, we want to do a right join. 

In [66]:
# Type your solution here
df1 = dta.drop(["violations"], axis='columns')
df2 = (dta.violations
 .str.split("|", expand=True)
 .unstack()
 .dropna()
 .reset_index(level=0, drop=True)
 .str.strip()
 .rename('violations')
 .to_frame()
 .sort_values('inspection_id'))

print(df1.index.name)
print(df2.index.name)

dta = df1.join(df2, how = 'right')
dta.head()

inspection_id
inspection_id


,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,zip,violations
inspection_id,,,,,,,,,,,,,,,
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,"41. PREMISES MAINTAINED FREE OF LITTER, UNNECE..."
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR..."
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO..."


In [59]:
%load solutions/join_violations.py

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,zip,violations
inspection_id,,,,,,,,,,,,,,,
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...


Now that we have a relatively clean DataFrame, let's ask a few more questions. 

First, how many unique violations do we have?

In [67]:
dta.shape

(117723, 15)

In [68]:
dta.violations.head()

inspection_id
44249    33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
44249    41. PREMISES MAINTAINED FREE OF LITTER, UNNECE...
44249    35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...
44249    18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
44249    34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...
Name: violations, dtype: object

In [69]:
dta.violations.unique().shape

(97972,)

Is this true? Do we really think there are this many violation numbers? Probably not. We can use the `str` accessor and some more munging to answer this. Here we pass a **regular expression** to `str.extract`. Extract expects a *capture group*, indicated by `()`. The regular expression `(\d+\)(?=\.)` means capture 1 or more (`+`) digits (`\d`) that is followed by (`(?=)`) a period `\.`. We escape the period because a plain `.` is a wildcard for any character.

In [70]:
(dta.violations
 .str.extract("(\d+)(?=\.)", expand=False)
 .astype(int))

inspection_id
44249      33
44249      41
44249      35
44249      18
44249      34
           ..
2102275    35
2102275    41
2102275    40
2102275    34
2102275    33
Name: violations, Length: 117723, dtype: int64

In [90]:
# # side note: rename a dataframe
# dta1 = dta.iloc[:5]

# (dta1.violations
#  .str.extract("(\d+)(?=\.)", expand=True)
#  .rename(columns = {0:'violations'}))

,violations
inspection_id,
44249,33
44249,41
44249,35
44249,18
44249,34


So how many unique violations do we have?

In [71]:
np.sort(
    dta.violations
        .str.extract("(\d+)(?=\.)", expand=False)
        .astype(int)
        .unique()
).shape

(45,)

Second, can we figure out how many times an establishment previously failed an inspection (within the sample we have)? How might we approach this? 

First, we want to restrict the data to just a single row for each inspection. Since we merged everything with the unstacked violations above, we'll need to use `drop_duplicates` to do this.

In [91]:
visits = dta.drop_duplicates(["address", "dba_name", "inspection_date"])

In [92]:
visits.head()

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,zip,violations
inspection_id,,,,,,,,,,,,,,,
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
44259,5501 N KEDZIE AVE,NORTHSIDE COLLEGE PREP,CHICAGO,NORTHSIDE COLLEGE PREP.,School,2010-05-12,Canvass,41.981352,46061.0,-87.708672,Pass,Risk 1 (High),IL,60625,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
44262,3132 N BROADWAY,NEW PEKING RESTAURANT,CHICAGO,NEW PEKING RESTAURANT,Restaurant,2010-06-09,Canvass,41.938648,31997.0,-87.644499,Pass,Risk 1 (High),IL,60657,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR..."
48225,2211 N OAK PARK AVE,SHRINERS HOSPITALS FOR CHILDREN,CHICAGO,SHRINERS HOSPITALS FOR CHILDREN,Hospital,2010-03-02,Canvass,41.920336,15783.0,-87.794994,Pass,Risk 1 (High),IL,60707,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO..."
52237,4140 N Marine DR,DISNEY MAGNET,CHICAGO,DISNEY MAGNET,School,2010-02-18,Canvass,41.956807,29401.0,-87.646118,Fail,Risk 1 (High),IL,60613,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...


We're going to rely on some pandas time-series functionality to do this, so we will need to ensure that the inspection dates are sorted within each group. GroupBy will preserve this.

In [93]:
visits = visits.sort_values(["address", "dba_name", "inspection_date"])

In [94]:
grouper = visits.groupby((visits.address, visits.dba_name))

/Users/firefreezing/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


Ok, we might ask, "now what?" Remember the trick to pull out groups? Let's use it to work with something we can think about.

In [95]:
group_key = list(grouper.groups.keys())[0]

In [96]:
group_key

('1 E 113TH ST ', 'V & J DAY CARE CENTER, INC')

In [97]:
group = grouper.get_group(group_key)

In [98]:
group

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,zip,violations
inspection_id,,,,,,,,,,,,,,,
419315,1 E 113TH ST,"V & J DAY CARE CENTER, INC",CHICAGO,"V & J DAY CARE CENTER, INC",Daycare Above and Under 2 Years,2010-10-08,Canvass,41.688848,1800876.0,-87.622894,Pass,Risk 1 (High),IL,60628,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...
537511,1 E 113TH ST,"V & J DAY CARE CENTER, INC",CHICAGO,"V & J DAY CARE CENTER, INC",Daycare Above and Under 2 Years,2011-07-28,License,41.688848,1800876.0,-87.622894,Pass,Risk 1 (High),IL,60628,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
545746,1 E 113TH ST,"V & J DAY CARE CENTER, INC",CHICAGO,"V & J DAY CARE CENTER, INC",Daycare Above and Under 2 Years,2012-01-13,Canvass,41.688848,1800876.0,-87.622894,Fail,Risk 1 (High),IL,60628,"41. PREMISES MAINTAINED FREE OF LITTER, UNNECE..."
545761,1 E 113TH ST,"V & J DAY CARE CENTER, INC",CHICAGO,"V & J DAY CARE CENTER, INC",Daycare Above and Under 2 Years,2012-01-20,Canvass Re-Inspection,41.688848,21702.0,-87.622894,Pass,Risk 1 (High),IL,60628,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR..."
1575903,1 E 113TH ST,"V & J DAY CARE CENTER, INC",CHICAGO,"V & J DAY CARE CENTER, INC",Daycare Above and Under 2 Years,2015-09-22,License,41.688848,2215538.0,-87.622894,Fail,Risk 1 (High),IL,60628,"41. PREMISES MAINTAINED FREE OF LITTER, UNNECE..."


In [99]:
group[['inspection_date', 'results']]

,inspection_date,results
inspection_id,,
419315,2010-10-08,Pass
537511,2011-07-28,Pass
545746,2012-01-13,Fail
545761,2012-01-20,Pass
1575903,2015-09-22,Fail


Since, we need this to be backwards looking, we will **shift** the data by one visit. Shifting will move the data around by either a number of periods or a frequency. In this case, we use a number of periods and shift forward by 1 period.

In [100]:
group.shift(1)[['inspection_date', 'results']]

,inspection_date,results
inspection_id,,
419315,NaT,NaN
537511,2010-10-08,Pass
545746,2011-07-28,Pass
545761,2012-01-13,Fail
1575903,2012-01-20,Pass


If we take the cumulative sum of this, we'll have an accurate picture of previous failures.

In [101]:
(group.shift(1).results == 'Fail').cumsum()

inspection_id
419315     0
537511     0
545746     0
545761     1
1575903    1
Name: results, dtype: int64

In [102]:
visit_num = grouper.apply(lambda df: (df.shift(1).results == 'Fail').cumsum())

In [103]:
visit_num.head(n=15)

address            dba_name                        inspection_id
1 E 113TH ST       V & J DAY CARE CENTER, INC      419315           0
                                                   537511           0
                                                   545746           0
                                                   545761           1
                                                   1575903          1
1 E 83RD ST        SUBWAY                          545430           0
                                                   1561815          0
1 E DELAWARE PL    MORE CUPCAKES                   1583338          0
                                                   1967048          0
                   STARBUCKS COFFEE # 13468        1567071          0
1 E JACKSON BLVD   Chartwells @ DePaul University  401321           0
                                                   1114427          0
                                                   1114923          0
                         

In [104]:
(visit_num.reset_index(level=[0, 1], drop=True)
 .to_frame()
 .rename(
    columns={'results': 'num_fails'}
))

,num_fails
inspection_id,
419315,0
537511,0
545746,0
545761,1
1575903,1
545430,0
1561815,0
1583338,0
1967048,0


In [106]:
dta.head()

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,zip,violations
inspection_id,,,,,,,,,,,,,,,
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,"41. PREMISES MAINTAINED FREE OF LITTER, UNNECE..."
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR..."
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
44249,960 W BELMONT AVE,Azha Restaurant,CHICAGO,Azha Restaurant Inc.,Restaurant,2010-01-21,Canvass Re-Inspection,41.940027,1334073.0,-87.653811,Pass,Risk 1 (High),IL,60657,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO..."


In [105]:
visit_num.head()

address        dba_name                    inspection_id
1 E 113TH ST   V & J DAY CARE CENTER, INC  419315           0
                                           537511           0
                                           545746           0
                                           545761           1
                                           1575903          1
Name: results, dtype: int64

In [107]:
visit_num.reset_index(level=[0, 1], drop=True).head()

inspection_id
419315     0
537511     0
545746     0
545761     1
1575903    1
Name: results, dtype: int64

In [108]:
(visit_num.reset_index(level=[0, 1], drop=True)
 .to_frame()
 .rename(
    columns={'results': 'num_fails'}
))

,num_fails
inspection_id,
419315,0
537511,0
545746,0
545761,1
1575903,1
545430,0
1561815,0
1583338,0
1967048,0


In [ ]:
dta.join((visit_num.reset_index(level=[0, 1], drop=True)
 .to_frame()
 .rename(
    columns={'results': 'num_fails'}
)))